In [2]:
import os
import shutil
import time
import pandas as pd
import sys
import numpy as np
import math
import random
from datetime import datetime
import locale
import glob
import re
import traceback
import pytz


start = time.time()
# Função para mover os arquivos de texto para a pasta 'Geração CAT'
def processar_loja(nome_arquivo, diretorio_principal):
    # Extrair o número da loja do nome do arquivo
    partes_nome = nome_arquivo.split('_')
    loja = partes_nome[1]
    periodo = int(partes_nome[4][1])

    # Construir o caminho para a pasta da loja
    pasta_loja = os.path.join(diretorio_principal, f"Loja_{loja}")
    print(pasta_loja)
    # Verificar se a pasta da loja existe
    if os.path.exists(pasta_loja) and os.path.isdir(pasta_loja):
        
        #### Tabela_2.1########################################################################################
        #Please store RAW FILE and GROUPO-OUTROS file in the file folder path
        file_folder_path = pasta_loja
        raw_file_name = f"loja_{loja}_tabela_2_p{periodo}_v1.csv"
        raw_file_path = os.path.join(diretorio_principal, raw_file_name)

        cnpj_file_name = "cnpjs.xlsx"
        cnpj_file_path = os.path.join(diretorio_referencias, cnpj_file_name)
        missing_cnpj_file_name=  f"Error_missing_cnpj_loja_{loja}.xlsx"
        missing_cnpj_file_path = os.path.join(diretorio_principal, missing_cnpj_file_name)

        final_file_name = f"loja_{loja}_tabela_2_1_p{periodo}_v1.csv"
        final_file_path = os.path.join(file_folder_path, final_file_name)
        
        #Import raw file , groupo
        df = pd.read_csv(raw_file_path, dtype={"CODIGO_BARRA": str,"Valor Base Cálculo ICMS Substituição Tributária":str, "NUM_ITEM":str, "CFOP":str,"COD_ITEM":str})

        #Please make sure the cnpj column is text
        cnpj= pd.read_excel(cnpj_file_path, dtype={"cnpj": str})
        
        ## remove duplicate in cnpj just in case
        cnpj.drop_duplicates(subset='cnpj', inplace=True)
        
        ## Add chave-item
        df['chave-item'] = df['CHV_DOC'] +'-'+ df['NUM_ITEM']
        
        ## Add cnpj
        df['cnpj'] = df['CHV_DOC'].apply(lambda x: x[6:20])
        
        dfa1= df.loc[df['CFOP'].str[0].isin(['1','2']),['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'COD_ITEM','IND_OPER', 'SUB_TIPO', 'QTD_CAT','cnpj','chave-item']]
        
        ## Merge with cpnj
        dfa1= dfa1.merge(cnpj, left_on='cnpj', right_on='cnpj', how='left')
        
        dfa1['grupo/outros'] = np.where(dfa1['CFOP']=='1411', 'grupo', dfa1['grupo/outros'])
        
        ## Checking missing cnpj, stop if there are missing cnpj in groupo and export missing cnpj to "missing cnpj.xlsx" file
        if dfa1.loc[dfa1['grupo/outros'].isna()].shape[0] > 0:
            dfa1.loc[dfa1['grupo/outros'].isna(),'cnpj'].drop_duplicates().to_excel(missing_cnpj_file_path,index=False)
            print("Por favor atualize o arquivo ""cnpj.xlsx"" com os cnpjs faltantes que estão no arquivo ""missing cnpj.xlsx"" ")
            sys.exit()
        else:
            pass
        
        
        dfa1.rename(columns={"grupo/outros":"tipo"}, inplace=True)
        
        dfa1['cod-tipo'] = dfa1['COD_ITEM'] + "-" + dfa1['tipo']
        
        #Take the unique cod-tipo
        dfa2 = dfa1[['COD_ITEM','cod-tipo','tipo']].drop_duplicates().sort_values(['cod-tipo','tipo'])
        
        #Extract the maximum cod item
        max_cod_item = df.loc[np.where(df['COD_ITEM'].str.isnumeric(), True, False ), 'COD_ITEM'].astype(float).astype(int).max() +1
        
        #new cod-item
        dfa2['tipo transform'] = dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount').cumsum().shift(1,fill_value=0) + max_cod_item
        
        dfa2['tipo2'] = np.where(dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount') >0 , dfa2['tipo'] + str(2), dfa2['tipo'] )
        dfa2['cod2'] = np.where(dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount') >0 , dfa2['tipo transform'].astype(str), dfa2['COD_ITEM'] )
        dfa2['cod2-tipo2'] = np.where(dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount') >0 , dfa2['cod2'].astype(str) + '-' + dfa2['tipo2'] , dfa2['cod-tipo'] )
        dfa2['novo'] = np.where(dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount') >0, True, False)
        
        dfa5 = dfa1.merge(dfa2[['cod-tipo', 'cod2-tipo2' , 'cod2', 'tipo2', 'novo']],left_on='cod-tipo', right_on='cod-tipo', how='left') \
        .groupby(['COD_ITEM', 'tipo', 'cod2-tipo2','cod2', 'tipo2', 'novo'], as_index=False, observed=True)['QTD_CAT'].sum() \
        .rename( columns={'QTD_CAT' : 'Total por forn'})
        
        #Only cod-novo not blank
        dfa6 = dfa5.loc[dfa5['novo'],['COD_ITEM', 'cod2']].drop_duplicates(subset='cod2')
        
        dfb1 = df.loc[df['CFOP'].str[0].isin(['5','6']),['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'COD_ITEM','IND_OPER', 'SUB_TIPO', 'QTD_CAT','chave-item']]
        
        #ADD qtd total entrada
        dfb1 = dfb1.merge( dfa5.loc[~dfa5['novo']].groupby(['COD_ITEM'],as_index=False, observed=True)['Total por forn'].sum()\
                        .rename(columns={'Total por forn':'qtd total entrada'}),\
                        left_on='COD_ITEM', right_on='COD_ITEM', how= 'left')
        dfb1['qtd total entrada'].fillna(0,inplace=True)
        
        dfb1 = dfb1.sort_values(['COD_ITEM'])
        dfb1['qtd acum saida'] = dfb1.groupby('COD_ITEM')['QTD_CAT'].transform('cumsum')
        
        dfb1['qtd total need'] = dfb1['qtd total entrada'] - dfb1['qtd acum saida']
        
        dfb1 = dfb1.merge(dfa6, left_on='COD_ITEM', right_on='COD_ITEM',how='left').rename(columns={'cod2':'cod2 test'})
        dfb1['cod2'] = np.where(dfb1['qtd total need']<0, dfb1['cod2 test'].fillna(dfb1['COD_ITEM']) , dfb1['COD_ITEM'] )
        
        dfb1 = dfb1.merge(dfa2[['cod2', 'tipo2']].drop_duplicates(), left_on='cod2', right_on='cod2', how='left')
        
        dfc1 =pd.concat([ dfb1[['chave-item', 'tipo2', 'cod2']] , \
                 dfa1.merge(dfa2[['cod-tipo','cod2-tipo2','cod2','tipo2']],left_on='cod-tipo', right_on='cod-tipo', how='left')[['chave-item','tipo2','cod2']] ] ).drop_duplicates(subset='chave-item')
        
        df_final = df.merge(dfc1, left_on= 'chave-item' , right_on ='chave-item', how='left')[[ \
        'CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod2','tipo2', 'COD_ITEM',
       'IND_OPER', 'SUB_TIPO', 'QTD_CAT', 'QTD_EFD', 'ICMS_TOT', 'VL_CONFR_0',
       'COD_LEGAL', 'ALIQUOTA', 'VALOR',
       'Valor Base Cálculo ICMS ST Retido Operação Anterior',
       'Valor Complementar', 'Valor ICMS Substituição Tributária',
       'Valor ICMS Operação', 'DESCRICAO', 'CODIGO_BARRA', 'UNIDADE', 'N C M',
       'CEST', 'CNPJ EMITENTE', 'CNPJ DESTINATARIO',
       'Valor Base Cálculo ICMS Substituição Tributária', 'vBCST', 'FONTE']]
        
        df_final.rename(columns={'cod2': 'cod novo', 
                                'tipo2': 'tipo'}, inplace=True)
        
        df_final['Valor Complementar'] = np.where(df_final['tipo'].isin(['outros', 'outros2']) ,
                                         0,
                                         df_final['Valor Complementar'])

        df_final['vBCST'] = df_final['vBCST'].fillna(0)
        df_final['ICMS_TOT'] = np.where((df_final['FONTE'] == 'entrada_sp') | (df_final['FONTE'] == 'entrada_outras'),
            np.where(df_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'] <= df_final['vBCST'],
                     np.maximum(df_final['Valor ICMS Operação'].fillna(0) + df_final['Valor ICMS Substituição Tributária'].fillna(0), (df_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'] + df_final['Valor Complementar'].fillna(0)) * df_final['ALIQUOTA'].fillna(0).astype(float)/100),
                     np.maximum(df_final['Valor ICMS Operação'].fillna(0) + df_final['Valor ICMS Substituição Tributária'].fillna(0), (df_final['vBCST'].astype(float).fillna(0) + df_final['Valor Complementar'].fillna(0)) * df_final['ALIQUOTA'].fillna(0).astype(float)/100)),
                                 np.nan)
        
        df_final.to_csv(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\loja_{loja}_tabela_2_1_p{periodo}_v1.csv')
        ######################################################################################################
        
        ####### Ficha 3 ######################################################################################
        df = pd.read_csv(os.path.join(pasta_loja, f'loja_{loja}_tabela_2_1_p{periodo}_v1.csv'), encoding='utf-8')
        df['cod novo'] = df['cod novo'].fillna('COD_ITEM')
        df['cod novo'] = df['cod novo'].astype(str)
        
        def calcular_ressarcimento(df):
            '''
            Função para cálculo do valor de ressarcimento
            para a loja e de outras informações para o
            arquivo TXT final

            '''

            df = df.drop('Unnamed: 0', axis=1)
            df = df.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                                       ascending=[True, True, True, True])

            # Criação de dataset para cálculo do ICMS inicial

            data = df[['cod novo', 'DATA', 'QTD_CAT', 'IND_OPER']]

            # Transformação da coluna 'DATA' para o tipo correto
            data['DATA'] = pd.to_datetime(data['DATA'], format='%Y-%m-%d')

            # Ordenação do dataset para cálculo correto do ICMS inicial

            data['IND_OPER'] = data['IND_OPER'].astype(int)
            data = data.sort_values(by=['cod novo','DATA', 'IND_OPER'])

            # Cáculo dos saldos das operações para definição da Quantidade Inicial

            data['OPER'] = np.where(data['IND_OPER'] == 0, data['QTD_CAT'], -data['QTD_CAT'])
            data['SALDO'] = data.groupby('cod novo')['OPER'].cumsum()

            # Criação da coluna para as informações da Quantidade Inicial

            min_saldo = data.groupby('cod novo')['SALDO'].transform('min')

            # Set 'QTD_INI' to the minimum 'SALDO' value if it's less than or equal to 0,
            # otherwise set it to 0
            data['QTD_INI'] = np.where(min_saldo <= 0, min_saldo.abs(), 0)

            # Get the first rows within each group
            first_rows = data.groupby('cod novo').head(1)

            # Set 'QTD_INI' to NaN for all rows except the last rows within each group
            data['QTD_INI'] = np.where(data.index.isin(first_rows.index), data['QTD_INI'], np.nan)

            # Update 'QTD_INI' with calculated values for the last rows
            data.loc[first_rows.index, 'QTD_INI'] = first_rows['QTD_INI']

            # Cálculo do ICMS inicial 

            data['SUB_TIPO'] = df['SUB_TIPO']
            data['ICMS_TOT_0'] = df['ICMS_TOT']
            data['VALOR_UNIT'] = np.where((data['SUB_TIPO'] == 1) & (data['IND_OPER'] == 0),
                                          data['ICMS_TOT_0'] / data['QTD_CAT'],
                                          np.nan)

            grouped_data = data.groupby('cod novo').agg({
                'VALOR_UNIT': 'mean',
                'QTD_INI': 'first'
            })

            # Vectorized calculation for icms_init
            grouped_data['ICMS_INI'] = grouped_data['VALOR_UNIT'] * (1 - 0.3 * np.random.random(size=len(grouped_data))) * grouped_data['QTD_INI']
            data = data.merge(grouped_data[['ICMS_INI']], how='left', left_on='cod novo', right_index=True)

            mask = data.duplicated(subset='cod novo', keep='first')

            # Assign 'ICMS_INI' values to NaN where mask is True, indicating it's not the first row for the 'COD_ITEM'
            data['ICMS_INI'] = np.where(mask, np.nan, data['ICMS_INI'])

            tabela_2 = df[['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod novo', 'IND_OPER', 'SUB_TIPO', 'QTD_CAT', 'ALIQUOTA', 'FONTE']]

            tabela_2['DATA'] = pd.to_datetime(tabela_2['DATA'], format='%Y-%m-%d')
            tabela_2 = tabela_2.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                                       ascending=[True, True, True, True])

            tabela_2['QTD_INI'] = data['QTD_INI']
            tabela_2['ICMS_INI'] = data['ICMS_INI']
            tabela_2['ICMS_INI'] = np.where((tabela_2['QTD_INI'] == 0) | (tabela_2['ICMS_INI'].isnull() & tabela_2['QTD_INI'].notnull()),
                                            0,
                                            tabela_2['ICMS_INI'])
            tabela_2 = tabela_2[['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod novo', 'IND_OPER', 
                                'SUB_TIPO', 'QTD_CAT','QTD_INI','ICMS_INI', 'ALIQUOTA', 'FONTE']]

            mask_condition1 = (tabela_2['IND_OPER'] == 0) & (tabela_2['SUB_TIPO'] == 1)
            mask_condition2 = (tabela_2['IND_OPER'] != 0) & (tabela_2['SUB_TIPO'] == -1)

            qtd_ent1_devolv_ent = list(np.where(mask_condition1, tabela_2['QTD_CAT'], 0) + np.where(mask_condition2, -tabela_2['QTD_CAT'], 0))
            tabela_2['QTD_ent1_devolv_ent'] = qtd_ent1_devolv_ent

            tabela_2['ICMS_TOT'] = df['ICMS_TOT']
            tabela_2['ICMS_TOT'] = tabela_2['ICMS_TOT'].fillna(0)

            mask = tabela_2['QTD_ent1_devolv_ent'] > 0

            tabela_2['ICMS_TOT_ent_unit'] = np.where(mask, tabela_2['ICMS_TOT'] / tabela_2['QTD_ent1_devolv_ent'], 0)

            tabela_2['ULT_ICMS_TOT_ent_unit'] = np.nan

            # Calculate the ULT_ICMS_TOT_ent_unit using vectorized operations
            values = np.where(
                (tabela_2['QTD_INI'] != 0) & 
                (tabela_2['cod novo'] == tabela_2['cod novo'].shift()) & 
                (tabela_2['QTD_ent1_devolv_ent'].shift() > 0),
                tabela_2['ICMS_TOT_ent_unit'].shift(),
                np.where(
                    (tabela_2['QTD_INI'] == 0),
                    0.01,
                    tabela_2['ICMS_INI'] / tabela_2['QTD_INI']
                )
            )

            tabela_2['tipo'] = df['tipo']

            # Assign values to the 'ULT_ICMS_TOT_ent_unit' column
            tabela_2['ULT_ICMS_TOT_ent_unit'] = values

            # Forward fill NaN values in 'ULT_ICMS_TOT_ent_unit' column
            tabela_2['ULT_ICMS_TOT_ent_unit'] = tabela_2['ULT_ICMS_TOT_ent_unit'].ffill()

            mask_condition1 = (tabela_2['IND_OPER'] == 1) & (tabela_2['SUB_TIPO'] == 1)
            mask_condition2 = (tabela_2['IND_OPER'] != 1) & (tabela_2['SUB_TIPO'] == -1)

            qtd_saida_1_devolv_saida_t = np.where(mask_condition1, tabela_2['QTD_CAT'], 0) + np.where(mask_condition2, -tabela_2['QTD_CAT'], 0)

            tabela_2['qtd_saida_1_devolv_saida'] = qtd_saida_1_devolv_saida_t

            tabela_2['QTD_SALDO'] = tabela_2.groupby('cod novo').apply(lambda x: (
            x['QTD_INI'].iloc[0] + (x['QTD_ent1_devolv_ent'] - x['qtd_saida_1_devolv_saida']).cumsum())).reset_index(level=0, drop=True)

            tabela_2['REF'] = tabela_2['DATA'].astype(str)
            tabela_2['REF'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m-%Y') for x in tabela_2['REF']]

            # Definição do primeiro valor de ICMS_SAIDA_UNI
            icms_saida_uni = [] 

            # Definição do primeiro valor de ULT_ICMS_SAIDA_UNI
            ult_icms_saida_uni = [] 

            # Definição do primeiro valor de ICMS_SAIDA
            icms_saida = [] 

            # Definição do primeiro valor de ICMS_TOT_SALDO
            icms_tot_saldo = [] 

            # Definição do primeiro valor de ICMS_TOT_1
            icms_tot_1 = []

            # Definição de todos os outros valores para cada campo acima
            for code in tabela_2['cod novo'].unique():

                icms_saida_uni_int = []
                ult_icms_saida_uni_int = []
                icms_saida_int = []
                icms_tot_saldo_int = []
                icms_tot_1_int = []
                qtd_saida_1_devolv_saida = list(tabela_2[tabela_2['cod novo'] == code]['qtd_saida_1_devolv_saida'])
                qtd_saldo = list(tabela_2[tabela_2['cod novo'] == code]['QTD_SALDO'])
                qtd_cat = list(tabela_2[tabela_2['cod novo'] == code]['QTD_CAT'])
                ref = list(tabela_2[tabela_2['cod novo'] == code]['REF'])

                for i,row in tabela_2[tabela_2['cod novo']==code].reset_index(drop=True).iterrows():

                # ICMS_SAIDA_UNI

                    if i == 0:
                        if qtd_saida_1_devolv_saida[0] > 0:
                            if row['QTD_INI'] == 0:
                                icms_saida_uni_int.append(0)
                            else:  
                                icms_saida_uni_int.append(row['ICMS_INI'] / row['QTD_INI'])
                        else:
                            icms_saida_uni_int.append(0)

                    else:
                        if (qtd_saida_1_devolv_saida[i] != 0) and (qtd_saldo[i-1] != 0):  
                            icms_saida_uni_int.append(icms_tot_saldo_int[i-1] / qtd_saldo[i-1])               
                        else:      
                            icms_saida_uni_int.append(0)

                # ULT_ICMS_SAIDA_UNI
                    if i== 0:
                        if row['QTD_INI'] == 0:
                            ult_icms_saida_uni_int.append(0.01)
                        else:           
                            ult_icms_saida_uni_int.append(
                                row['ICMS_INI'] / row['QTD_INI'] if row['QTD_INI'] != 0 else float("nan"))
                    else:
                        if qtd_saida_1_devolv_saida[i-1] > 0:
                            ult_icms_saida_uni_int.append(icms_saida_uni_int[i-1])
                        else:
                            ult_icms_saida_uni_int.append(ult_icms_saida_uni_int[i-1])


                # ICMS_SAIDA
                    if qtd_saida_1_devolv_saida[i] < 0:
                        icms_saida_int.append(qtd_saida_1_devolv_saida[i]*ult_icms_saida_uni_int[i])
                    else:
                        icms_saida_int.append(icms_saida_uni_int[i]*qtd_saida_1_devolv_saida[i])

                # ICMS_TOT_1
                    if i == 0:
                        if (row['QTD_ent1_devolv_ent'] < 0) & (row['QTD_CAT'] == row['QTD_INI']) :
                            icms_tot_1_int.append(-row['ICMS_INI'])
                        else:
                            icms_tot_1_int.append(row['ICMS_TOT'])
                    else:
                        if row['QTD_ent1_devolv_ent'] < 0:
                            if (qtd_cat[i] == qtd_saldo[i-1]):
                                icms_tot_1_int.append(-1*icms_tot_saldo_int[i-1])
                            else:
                                icms_tot_1_int.append(max(row['QTD_ent1_devolv_ent'] * row['ULT_ICMS_TOT_ent_unit'], -1*icms_tot_saldo_int[i-1]))
                        else:
                            icms_tot_1_int.append(row['ICMS_TOT']) 

                # ICMS_TOT_SALDO
                    if i == 0:
                        icms_tot_saldo_int.append(row['ICMS_INI'] + icms_tot_1_int[i] - icms_saida_int[i])
                    else:
                        icms_tot_saldo_int.append(icms_tot_saldo_int[i-1] + icms_tot_1_int[i] - icms_saida_int[i])


                icms_saida_uni += icms_saida_uni_int
                ult_icms_saida_uni += ult_icms_saida_uni_int
                icms_saida += icms_saida_int
                icms_tot_saldo += icms_tot_saldo_int
                icms_tot_1 += icms_tot_1_int

            tabela_2['ICMS_SAIDA_UNI'] = icms_saida_uni
            tabela_2['ULT_ICMS_SAIDA_UNI'] = ult_icms_saida_uni
            tabela_2['ICMS_SAIDA'] = icms_saida
            tabela_2['ICMS_TOT_SALDO'] = icms_tot_saldo
            tabela_2['ICMS_TOT_SALDO'] = np.where((tabela_2['ICMS_TOT_SALDO'] < 0.01) & (tabela_2['ICMS_TOT_SALDO'] > -0.01),
                                                 0,
                                                 tabela_2['ICMS_TOT_SALDO'])
            tabela_2['ICMS_TOT_1'] = icms_tot_1

            tabela_2['ICMS_TOT_PCAT'] = np.where((tabela_2['qtd_saida_1_devolv_saida'] < 0),
                                             np.abs(tabela_2['ICMS_SAIDA_UNI']),
                                             np.where((tabela_2['QTD_ent1_devolv_ent'] != 0),
                                                      np.abs(tabela_2['ICMS_TOT_1']),
                                                      np.nan))

            if tabela_2[tabela_2['ICMS_TOT_PCAT'] < 0].shape[0] > 0:
                mensagem('EXISTE ICMS TOT PCAT NEGATIVO. FAVOR CHECAR!!')
                print(mensagem)

            tabela_2['VLR_CONF_0'] = df['VL_CONFR_0']

            mask = tabela_2['qtd_saida_1_devolv_saida'] > 0
            vlr_confr_unit = np.where(mask, tabela_2['VLR_CONF_0']/tabela_2['qtd_saida_1_devolv_saida'], np.nan)

            tabela_2['VLR_CONFR_UNIT'] = vlr_confr_unit

            ult_vlr_confr_unit = [0]
            for i in range(1, tabela_2.shape[0]):
                if tabela_2.iloc[i]['cod novo'] != tabela_2.iloc[i-1]['cod novo']:
                    ult_vlr_confr_unit.append(0)
                else:
                    if math.isnan(tabela_2.iloc[i-1]['VLR_CONFR_UNIT']):
                        ult_vlr_confr_unit.append(ult_vlr_confr_unit[i-1])
                    else:
                        ult_vlr_confr_unit.append(tabela_2.iloc[i-1]['VLR_CONFR_UNIT'])

            tabela_2['ULT_VLR_CONFR_UNIT'] = ult_vlr_confr_unit

            tabela_2['VLR_CONFR_1'] = np.where(tabela_2['qtd_saida_1_devolv_saida'] > 0,
                                           tabela_2['VLR_CONF_0'],
                                           np.where(tabela_2['qtd_saida_1_devolv_saida'] < 0,
                                                    tabela_2['ULT_VLR_CONFR_UNIT'] * tabela_2['qtd_saida_1_devolv_saida'],
                                                    0))

            tabela_2['VLR_CONFR_1'] = np.where(tabela_2['VLR_CONFR_1'] <= 0.01,
                                                      0.01,
                                                      tabela_2['VLR_CONFR_1'])

            tabela_2['COD_LEGAL'] = df['COD_LEGAL']

            mask = np.abs(tabela_2['ICMS_SAIDA']) > np.abs(tabela_2['VLR_CONFR_1'])
            vlr_ressarc = np.where(tabela_2['COD_LEGAL'] == 1, (np.where(mask, tabela_2['ICMS_SAIDA']-tabela_2['VLR_CONFR_1'], 0)),0)

            tabela_2['VLR_RESSARCIMENTO'] = vlr_ressarc

            mask = np.abs(tabela_2['ICMS_SAIDA']) < np.abs(tabela_2['VLR_CONFR_1'])
            vlr_compl = np.where(tabela_2['COD_LEGAL'] == 1, np.where(mask, -tabela_2['ICMS_SAIDA']+tabela_2['VLR_CONFR_1'], 0),0)

            tabela_2['VLR_COMPLEMENTO'] = vlr_compl

            tabela_2['COD_LEGAL_PCAT'] = np.where((tabela_2['COD_LEGAL'] == 1) & (tabela_2['VLR_RESSARCIMENTO'] != 0) & (tabela_2['tipo'] != 'outros2') & (tabela_2['ALIQUOTA'] != 0), 1, 0)
            tabela_2.loc[pd.isna(tabela_2['COD_LEGAL']), 'COD_LEGAL_PCAT'] = np.nan

            tabela_2['VLR_CONFR_PCAT'] = np.where(tabela_2['COD_LEGAL_PCAT'] == 1, np.abs(tabela_2['VLR_CONFR_1']), np.nan)
            if tabela_2[(tabela_2['COD_LEGAL'] == 1) & (tabela_2['VLR_CONFR_PCAT'] <= 0)].shape[0] > 0:
                mensagem = 'Há valores de confronto inconsistentes. Favor checar.'
                print(mensagem)
                with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                    arquivo.write(mensagem)
                sys.exit()

            mask = (tabela_2['cod novo'].shift(-1) != tabela_2['cod novo']) | (tabela_2['REF'].shift(-1) != tabela_2['REF'])
            saldo_final_mes_qtd = np.where(mask, tabela_2['QTD_SALDO'], np.nan)

            tabela_2['SALDO_FINAL_MES_QTD'] = saldo_final_mes_qtd

            mask = (tabela_2['cod novo'].shift(-1) != tabela_2['cod novo']) | (tabela_2['REF'].shift(-1) != tabela_2['REF'])
            saldo_final_mes_icms = np.where(mask, tabela_2['ICMS_TOT_SALDO'], np.nan)

            tabela_2['SALDO_FINAL_MES_ICMS'] = saldo_final_mes_icms

            tabela_2['VALOR'] = df['VALOR']
            tabela_2['Valor Base Cálculo ICMS ST Retido Operação Anterior'] = df['Valor Base Cálculo ICMS ST Retido Operação Anterior']
            tabela_2['Valor Complementar'] = df['Valor Complementar']
            tabela_2['vBCST'] = df['vBCST']
            tabela_2['Valor ICMS Substituição Tributária'] = df['Valor ICMS Substituição Tributária']
            tabela_2['Valor ICMS Operação'] = df['Valor ICMS Operação']
            tabela_2['CNPJ EMITENTE'] = df['CNPJ EMITENTE']
            tabela_2['COD_ITEM'] = df['COD_ITEM']


            tabela_2.rename(columns={'ICMS_TOT': 'ICMS_TOT_0',}, inplace=True)
            tabela_2['ICMS_TOT'] = df['ICMS_TOT']

            tabela_2 = tabela_2[['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod novo', 'IND_OPER', 'SUB_TIPO', 'QTD_CAT',
                                'QTD_INI', 'ICMS_INI', 'QTD_ent1_devolv_ent', 'ICMS_TOT_0', 'ICMS_TOT_ent_unit',
                                'ULT_ICMS_TOT_ent_unit', 'ICMS_TOT_1', 'qtd_saida_1_devolv_saida', 'ICMS_SAIDA_UNI',
                                'ULT_ICMS_SAIDA_UNI', 'ICMS_SAIDA', 'ICMS_TOT_PCAT', 'VLR_CONF_0', 'VLR_CONFR_UNIT',
                                'ULT_VLR_CONFR_UNIT', 'VLR_CONFR_1', 'QTD_SALDO', 'ICMS_TOT_SALDO', 'VLR_RESSARCIMENTO',
                                'VLR_COMPLEMENTO', 'COD_LEGAL', 'COD_LEGAL_PCAT', 'VLR_CONFR_PCAT', 'SALDO_FINAL_MES_QTD',
                                'SALDO_FINAL_MES_ICMS', 'ALIQUOTA', 'VALOR', 'Valor Base Cálculo ICMS ST Retido Operação Anterior',
                                'Valor Complementar', 'Valor ICMS Substituição Tributária', 'Valor ICMS Operação', 
                                'ICMS_TOT', 'COD_ITEM', 'CNPJ EMITENTE', 'tipo', 'vBCST', 'FONTE']]

            return tabela_2
        
        tabela_2 = calcular_ressarcimento(df)
        
        meta_por_loja = pd.read_excel(os.path.join(diretorio_referencias, 'GTI_12.xlsx'))
        try:
            meta = meta_por_loja[(meta_por_loja['Loja'] == loja) & (meta_por_loja['periodo'] == f'p{periodo}')]['ESTIMATI3'].values[0]
        except:
            meta = 0.085
        
        def gti_pra_cima(tabela_2, meta):
    
            '''
            Função para aplicação de GTI para lojas
            com ressarcimento abaixo da porcentagem estabelecida
            como meta

            '''

            valor = tabela_2[tabela_2['CFOP'] == 5405]['VALOR'].sum()
            ressarc = tabela_2['VLR_RESSARCIMENTO'].sum()
            fator = 0.1

            # 1ª Iteração

            tabela_2['cod novo'] = tabela_2['cod novo'].astype(str)
            pivot_ressarc = tabela_2[tabela_2['tipo'] == 'grupo'].pivot_table(index='cod novo', values='VLR_RESSARCIMENTO', aggfunc='sum')
            pivot_ressarc = pivot_ressarc[pivot_ressarc['VLR_RESSARCIMENTO'] > 0]
            pivot_ressarc = pivot_ressarc.sort_values(by='VLR_RESSARCIMENTO', ascending=False)  

            incr_max_ref = pivot_ressarc.iloc[54].values[0]
            incr_max = pivot_ressarc.iloc[55:]
            incr_max['Diferenças'] = incr_max_ref - incr_max['VLR_RESSARCIMENTO']
            incr_max = incr_max.sort_values(by='cod novo')

            qtd_saida = tabela_2[tabela_2['CFOP'] == 5405].pivot_table(index='cod novo', values='QTD_CAT', aggfunc='sum')
            incr_max_unit = incr_max.merge(qtd_saida, on='cod novo', how='left')
            incr_max_unit['Incremento maximo unitario'] = incr_max_unit['Diferenças'] / incr_max_unit['QTD_CAT']
            incr_max_unit = incr_max_unit.reset_index()

            tabela_2 = tabela_2.merge(incr_max_unit[['cod novo', 'Incremento maximo unitario']], on='cod novo', how='left')
            tabela_2['Incremento maximo unitario'] = np.where(tabela_2['ICMS_TOT'].fillna(0) == 0,
                                                         0,
                                                         tabela_2['Incremento maximo unitario'])

            tabela_2['Incremento maximo unitario'] = tabela_2['Incremento maximo unitario'].fillna(0)

            tabela_2['ICMS_TOT_FINAL'] = np.where(tabela_2['ICMS_TOT'].fillna(0) != 0,
                                             tabela_2['ICMS_TOT'] + (tabela_2['Incremento maximo unitario']*fator)*tabela_2['QTD_CAT'],
                                             0)

            tabela_2['bc ST op ant'] = np.where(tabela_2['ALIQUOTA'] != 0,
                                           tabela_2['ICMS_TOT_FINAL'] / (tabela_2['ALIQUOTA']/100),
                                           tabela_2['Valor Base Cálculo ICMS ST Retido Operação Anterior'])

            tabela_2['bc ST op ant comple'] = tabela_2['bc ST op ant'] - tabela_2['Valor Base Cálculo ICMS ST Retido Operação Anterior']

            df_final = df.copy()
            df_final = df_final.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                                           ascending=[True, True, True, True]).reset_index().drop(['index'], axis=1)

            df_final['bc ST op ant comple'] = np.where(tabela_2['tipo'] == 'grupo',
                                                       tabela_2['bc ST op ant comple'],
                                                       tabela_2['Valor Complementar'])

            df_final.rename(columns={'ICMS_TOT': 'ICMS_TOT_ORIG',
                                    'Valor Complementar': 'Complementar Original',
                                    'bc ST op ant comple': 'Valor Complementar'}, inplace=True)

            df_final['ICMS_TOT'] = np.where(df_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'] <= df_final['vBCST'],
                     np.maximum(df_final['Valor ICMS Operação'].fillna(0) + df_final['Valor ICMS Substituição Tributária'].fillna(0), (df_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'] + df_final['Valor Complementar'].fillna(0)) * df_final['ALIQUOTA'].fillna(0).astype(float)/100),
                     np.maximum(df_final['Valor ICMS Operação'].fillna(0) + df_final['Valor ICMS Substituição Tributária'].fillna(0), (df_final['vBCST'] + df_final['Valor Complementar'].fillna(0)) * df_final['ALIQUOTA'].fillna(0).astype(float)/100))


            tabela_2_final = calcular_ressarcimento(df_final)

            # 2ª Iteração
            ressarc_tot_0_1 = tabela_2_final['VLR_RESSARCIMENTO'].sum()
            delta_fator_0_1 = ressarc_tot_0_1 - ressarc
            meta = meta
            meta_ressarc = valor * meta
            meta_delta = meta_ressarc - ressarc
            meta_fator_0_1 = meta_delta/delta_fator_0_1
            meta_fator = meta_fator_0_1*0.1*0.95

            tabela_2_final['cod novo'] = tabela_2_final['cod novo'].astype(str)
            pivot_ressarc_final = tabela_2_final[tabela_2_final['tipo'] == 'grupo'].pivot_table(index='cod novo', values='VLR_RESSARCIMENTO', aggfunc='sum')
            pivot_ressarc_final = pivot_ressarc_final[pivot_ressarc_final['VLR_RESSARCIMENTO'] > 0]
            pivot_ressarc_final = pivot_ressarc_final.sort_values(by='VLR_RESSARCIMENTO', ascending=False)


            incr_max_ref_final = pivot_ressarc_final.iloc[54].values[0]
            incr_max_final = pivot_ressarc_final.iloc[55:]
            incr_max_final['Diferenças'] = incr_max_ref_final - incr_max_final['VLR_RESSARCIMENTO']
            incr_max_final = incr_max_final.sort_values(by='cod novo')

            qtd_saida_final = tabela_2_final[tabela_2_final['CFOP'] == 5405].pivot_table(index='cod novo', values='QTD_CAT', aggfunc='sum')
            incr_max_unit_final = incr_max_final.merge(qtd_saida_final, on='cod novo', how='left')
            incr_max_unit_final['Incremento maximo unitario'] = incr_max_unit_final['Diferenças'] / incr_max_unit_final['QTD_CAT']

            incr_max_unit_final = incr_max_unit_final.reset_index()

            tabela_2_final['ICMS_TOT_ORIG'] = df_final['ICMS_TOT_ORIG']
            tabela_2_final = tabela_2_final.merge(incr_max_unit_final[['cod novo', 'Incremento maximo unitario']], on='cod novo', how='left')
            tabela_2_final['Incremento maximo unitario'] = np.where(tabela_2_final['ICMS_TOT_ORIG'].fillna(0) == 0,
                                                         0,
                                                         tabela_2_final['Incremento maximo unitario'])
            tabela_2_final['Incremento maximo unitario'] = tabela_2_final['Incremento maximo unitario'].fillna(0)

            tabela_2_final['ICMS_TOT_FINAL'] = np.where(tabela_2_final['ICMS_TOT_ORIG'].fillna(0) != 0,
                                             tabela_2_final['ICMS_TOT_ORIG'] + (tabela_2_final['Incremento maximo unitario']*meta_fator)*tabela_2_final['QTD_CAT'],
                                             0)

            tabela_2_final['bc ST op ant'] = np.where(tabela_2_final['ALIQUOTA'] != 0,
                                           tabela_2_final['ICMS_TOT_FINAL'] / (tabela_2_final['ALIQUOTA']/100),
                                           tabela_2_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'])

            tabela_2_final['bc ST op ant comple'] = tabela_2_final['bc ST op ant'] - tabela_2_final['Valor Base Cálculo ICMS ST Retido Operação Anterior']


            df_fim = df.copy()
            df_fim = df_fim.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                                           ascending=[True, True, True, True]).reset_index().drop(['index'], axis=1)

            df_fim['bc ST op ant comple'] = np.where(tabela_2_final['tipo'] == 'grupo',
                                                       tabela_2_final['bc ST op ant comple'],
                                                       tabela_2_final['Valor Complementar'])

            df_fim.rename(columns={'ICMS_TOT': 'ICMS_TOT_ORIG',
                                    'Valor Complementar': 'Complementar Original',
                                    'bc ST op ant comple': 'Valor Complementar'}, inplace=True)

            df_fim['ICMS_TOT'] = np.where(df_fim['Valor Base Cálculo ICMS ST Retido Operação Anterior'] <= df_fim['vBCST'],
                     np.maximum(df_fim['Valor ICMS Operação'].fillna(0) + df_fim['Valor ICMS Substituição Tributária'].fillna(0), (df_fim['Valor Base Cálculo ICMS ST Retido Operação Anterior'] + df_fim['Valor Complementar'].fillna(0)) * df_fim['ALIQUOTA'].fillna(0).astype(float)/100),
                     np.maximum(df_fim['Valor ICMS Operação'].fillna(0) + df_fim['Valor ICMS Substituição Tributária'].fillna(0), (df_fim['vBCST'] + df_fim['Valor Complementar'].fillna(0)) * df_fim['ALIQUOTA'].fillna(0).astype(float)/100))

            tabela_2_fim = calcular_ressarcimento(df_fim)

            return tabela_2_fim

        def gti_pra_baixo(tabela_2, meta):
            '''
            Função para aplicação de GTI para lojas
            com ressarcimento acima da porcentagem estabelecida
            como meta

            '''

            ressarc = tabela_2['VLR_RESSARCIMENTO'].sum()
            meta_ressarc = tabela_2[tabela_2['CFOP'] == 5405]['VALOR'].sum() * meta
            diferenca = ressarc-meta_ressarc
            pivot_ressarc = tabela_2.pivot_table(index='cod novo', values='VLR_RESSARCIMENTO', aggfunc='sum')
            pivot_ressarc = pivot_ressarc[pivot_ressarc['VLR_RESSARCIMENTO'] > 0].sort_values(by='VLR_RESSARCIMENTO',
                                                                                             ascending=True)
            soma = 0
            i = 0
            cods = []
            while i < len(pivot_ressarc) and (soma + pivot_ressarc['VLR_RESSARCIMENTO'].iloc[i]) <= diferenca:
                soma += pivot_ressarc['VLR_RESSARCIMENTO'].iloc[i]
                cods.append(pivot_ressarc.index[i])
                i += 1

            df['COD_LEGAL'] = np.where(df['cod novo'].isin(cods),
                                      np.where(df['COD_LEGAL'].isnull(), df['COD_LEGAL'], 0),
                                      df['COD_LEGAL'])

            df['VL_CONFR_0'] = np.where(df['cod novo'].isin(cods),
                                       np.nan,
                                       df['VL_CONFR_0'])

            tabela_2_final = calcular_ressarcimento(df)

            return tabela_2_final
        
        
        if (tabela_2['VLR_RESSARCIMENTO'].sum() / tabela_2[tabela_2['CFOP'] == 5405]['VALOR'].sum()) < meta:
            tabela_2_def = gti_pra_cima(tabela_2, meta)
    
        elif (tabela_2['VLR_RESSARCIMENTO'].sum() / tabela_2[tabela_2['CFOP'] == 5405]['VALOR'].sum()) > meta:
            print('ATENÇÃO. LOJA COM PORCENTAGEM DE RESSARCIMENTO ACIMA DA MARGEM')
            print('+'*50)
            print('\n')
            print('REPETINDO. ATENÇÃO. LOJA COM PORCENTAGEM DE RESSARCIMENTO ACIMA DA MARGEM')
            tabela_2_def = gti_pra_baixo(tabela_2, meta)
            
        # Criação das funções para geração dos arquivos Excel

        def gerar_excel_lojas_pequenas(tabela_2):

            path = os.path.join(pasta_loja, f'ficha_3_loja_{loja}_p{periodo}_v1.xlsx')
            with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
                tabela_2.to_excel(writer, sheet_name='ficha3', index=False)

        def gerar_excel_lojas_grandes(tabela_2):
            '''
            Função para exportação da Ficha 3 em arquivo excel
            para o caso de o arquivo de ficha 3 da loja tenha
            mais de 1 milhão de linhas

            '''

            cods = list(tabela_2['cod novo'].unique())
            l = len(cods)

            fat_div = 2

            while True:

                cods_p1 = cods[:int(l/fat_div)]
                cods_p2 = cods[int(l/fat_div):]

                tabela_2_1 = tabela_2[tabela_2['cod novo'].isin(cods_p1)]
                tabela_2_2 = tabela_2[tabela_2['cod novo'].isin(cods_p2)]

                if tabela_2_1.shape[0] > 1000000:
                    fat_div += 1

                else:
                    break

            # Geração do arquivo final referente ao dataframe 1
            path = f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_parte1_v1.xlsx'
            with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
                tabela_2_1.to_excel(writer, sheet_name='ficha3', index=False)

            # Geração do arquivo final referente ao dataframe 2
            path = f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_parte2_v1.xlsx'
            with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
                tabela_2_2.to_excel(writer, sheet_name='ficha3', index=False)
                
        
        if tabela_2_def.shape[0] > 1000000:
            print('+'*50)
            print('ATENÇÃO, LOJA GRANDE. ATENÇÃO, LOJA GRANDE')
            print('+'*50)

            gerar_excel_lojas_grandes(tabela_2_def)

        else:
            gerar_excel_lojas_pequenas(tabela_2_def)
            
        tabela_2_def['REF'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m') for x in tabela_2_def['DATA'].astype(str)]
        pivot = tabela_2_def.pivot_table(index='REF', values=['VLR_RESSARCIMENTO', 'VLR_COMPLEMENTO'], aggfunc='sum').sort_values(by='REF')
        pivot_fat = tabela_2_def[tabela_2_def['CFOP'] == 5405].pivot_table(index='REF', values='VALOR', aggfunc='sum').sort_values(by='REF')
        fuso_horario_brasilia = pytz.timezone('America/Sao_Paulo')
        data_geracao_brasilia = datetime.now(fuso_horario_brasilia).strftime('%Y-%m-%d %H:%M:%S')

        pivot['VALOR'] = pivot_fat['VALOR']
        pivot['Data'] = data_geracao_brasilia
        cnpj = tabela_2_def[tabela_2_def['FONTE'] == 'dfe']['CHV_DOC'].str.slice(6,20).unique()
        pivot['CNPJ'] = list(cnpj)*len(pivot)
        
        pivot.rename(columns={'REF': 'ANO-MES'})
        pivot.to_excel(os.path.join(pasta_loja, f'relatorio_mensal_ressarcimento_complemento_{loja}_p{periodo}_v1.xlsx'))
        
        ########################################################################################################################
        
        ##### Geração 1050 #####################################################################################################
        
        if tabela_2_def.shape[0] > 1000000:
                       
            df_1 = pd.read_excel(os.path.join(pasta_loja, f'ficha_3_loja_{loja}_p{periodo}_parte1_v1.xlsx'))
            df_2 = pd.read_excel(os.path.join(pasta_loja, f'ficha_3_loja_{loja}_p{periodo}_parte2_v1.xlsx'))
            df = pd.concat([df_1, df_2])
                       
        else:
            df = pd.read_excel(os.path.join(pasta_loja, f'ficha_3_loja_{loja}_p{periodo}_v1.xlsx'))

        
        df = df[['DATA','cod novo', 'QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
        
        df['REF'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y%m') for x in df['DATA'].astype(str)]
        df['REF'] = df['REF'].astype(str)
        
        grouped = df.groupby(['cod novo', 'REF']).agg(
            QTD_INI=('QTD_INI', 'first'),
            ICMS_INI=('ICMS_INI', 'first'),
            SALDO_FINAL_MES_QTD=('SALDO_FINAL_MES_QTD', 'last'),
            SALDO_FINAL_MES_ICMS=('SALDO_FINAL_MES_ICMS', 'last')
        ).reset_index()

        df_1050 = grouped.copy()
                               
        
        df_1050['QTD_INI'] = np.where(df_1050['QTD_INI'].isnull(),
                             df_1050['SALDO_FINAL_MES_QTD'].shift(),
                             df_1050['QTD_INI'])

        df_1050['ICMS_INI'] = np.where(df_1050['ICMS_INI'].isnull(),
                                     df_1050['SALDO_FINAL_MES_ICMS'].shift(),
                                     df_1050['ICMS_INI'])

        df_1050['ICMS_INI'] = np.where(df_1050['QTD_INI'] == 0,
                                      0,
                                       df_1050['ICMS_INI'])
                               
        if len(df_1050[df_1050['SALDO_FINAL_MES_ICMS'] < 0]) > 0:
            mensagem1 = 'ATENÇAO. EXISTE SALDO DE ICMS NEGATIVO'
            mensagem2 = 'ATENÇÃO. FAVOR CHECAR SOBRE VALOR DE SALDO DE ICMS NEGATIVO'
            print(mensagem1)
            print(mensagem2)
            with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                arquivo.write(mensagem1)
                arquivo.write(mensagem2)
            sys.exit()  
            
        if len(df_1050[df_1050['SALDO_FINAL_MES_QTD'] < 0]) > 0:
            mensagem1 = 'ATENÇAO. EXISTE SALDO DE QUANTIDADE NEGATIVO'
            mensagem2 = 'ATENÇÃO. FAVOR CHECAR SOBRE VALOR DE SALDO DE QUANTIDADE NEGATIVO'
            print(mensagem1)
            print(mensagem2)
            with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                arquivo.write(mensagem1)
                arquivo.write(mensagem2)
            sys.exit()
            
        if df_1050[(df_1050['QTD_INI'] == 0) & (df_1050['ICMS_INI'] != 0)].shape[0] > 0:
            mensagem = 'ATENÇÂO. ICMS INICIAL DIFERENTE DE ZERO PARA QUANTIDADE INICIAL IGUAL A ZERO. FAVOR CHECAR'
            with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                arquivo.write(mensagem)
            sys.exit()
            
        if df_1050[(df_1050['SALDO_FINAL_MES_QTD'] == 0) & (df_1050['SALDO_FINAL_MES_ICMS'] != 0)].shape[0] > 0:
            mensagem = 'ATENÇÂO. ICMS FINAL DIFERENTE DE ZERO PARA QUANTIDADE FINAL IGUAL A ZERO. FAVOR CHECAR'
            with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                arquivo.write(mensagem)
            sys.exit()
            
        df_1050.to_excel(os.path.join(pasta_loja, f'1050_loja_{loja}_p{periodo}_v1.xlsx'))  
        #####################################################################################################################
                        
        ###### Geração TXT ##################################################################################################
                        
        if tabela_2_def.shape[0] > 1000000:
            planilha1 = pd.read_excel(os.path.join(pasta_loja, f'ficha_3_loja_{loja}_p{periodo}_parte1_v1.xlsx'))
            planilha2 = pd.read_excel(os.path.join(pasta_loja, f'ficha_3_loja_{loja}_p{periodo}_parte2_v1.xlsx'))

            planilha = pd.concat([planilha1, planilha2])
                        
        else:
            planilha = pd.read_excel(os.path.join(pasta_loja, f'ficha_3_loja_{loja}_p{periodo}_v1.xlsx'))
                               
        planilha = planilha.reset_index()
        planilha['DATA'] = pd.to_datetime(planilha['DATA'], format='%Y-%m-%d')        
                        
        tabela_2 = pd.read_csv(os.path.join(pasta_loja, f'loja_{loja}_tabela_2_1_p{periodo}_v1.csv'))
        tabela_2 = tabela_2.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                                   ascending=[True, True, True, True])
                               
                               
        planilha = planilha.merge(tabela_2[['CHV_DOC', 'NUM_ITEM', 'DESCRICAO', 'CODIGO_BARRA', 'UNIDADE', 'N C M', 'CEST',
                'CNPJ DESTINATARIO']],
            on=['CHV_DOC', 'NUM_ITEM'], how='left') 
        
        planilha['cod novo'] = planilha['cod novo'].astype(str)
                        
        planilha['DESCRICAO'] = planilha['DESCRICAO'].str.replace('"', "'")
                        
        planilha['N C M'] = pd.to_numeric(planilha['N C M'])
        planilha['N C M'] = planilha['N C M'].astype(str).str.replace(r'\.0$', '', regex=True)

        planilha['CEST'] = pd.to_numeric(planilha['CEST'])
        planilha['CEST'] = planilha['CEST'].astype('Int64').astype(str).replace('<NA>', np.nan)
        planilha['CEST'] = planilha['CEST'].astype(str).str.replace(r'\.0$', '', regex=True)

        planilha['CNPJ EMITENTE'] = pd.to_numeric(planilha['CNPJ EMITENTE'])
        planilha['CNPJ EMITENTE'] = planilha['CNPJ EMITENTE'].astype('Int64').astype(str).replace('<NA>', np.nan)

        planilha['QTD_CAT'] = planilha['QTD_CAT'].astype(str) + ',000'

        planilha['COD_LEGAL_PCAT'] = pd.to_numeric(planilha['COD_LEGAL_PCAT'])
        planilha['COD_LEGAL_PCAT'] = planilha['COD_LEGAL_PCAT'].astype('Int64').astype(str).replace('<NA>', np.nan)

        planilha['CFOP'] = pd.to_numeric(planilha['CFOP'])
        planilha['CFOP'] = planilha['CFOP'].astype('Int64').astype(str).replace('<NA>', np.nan)

        planilha['UNIDADE'] = planilha['UNIDADE'].str.strip()

        def format_number(value):
            return '{:,.2f}'.format(value).replace(',', '.')
                        
        planilha = planilha.drop_duplicates()
                        
        planilha['CODIGO_BARRA'] = np.where(pd.notnull(planilha['CODIGO_BARRA']), planilha['CODIGO_BARRA'].astype(str).str.replace(r'\.0$', '', regex=True),
                                   np.nan)
                        
        mes_ano = []
        for date in planilha['DATA']:
            mes_ano.append(str(date)[5:7] + str(date)[0:4])

        planilha['Data'] = mes_ano
        planilha['Data'].unique()
                        
        planilha.replace('nan', np.nan, inplace=True)
        
        planilha['CEST'] = planilha['CEST'].replace('0', pd.NA)

        # Second, add leading '0' to values with length equal to 6
        planilha.loc[planilha['CEST'].str.len() == 6, 'CEST'] = '0' + planilha['CEST']
                        
        planilha['N C M'] = planilha['N C M'].apply(lambda x: str(x).zfill(8) if len(str(x)) < 8 else str(x))
                        
        planilha['CODIGO_BARRA'] = planilha['CODIGO_BARRA'].replace('SEM GTIN      ', np.nan)
                        
        # Geração do dataset base para geração da 1100

        cols_1100 = ['CHV_DOC', 'DATA', 'NUM_ITEM', 'IND_OPER', 'cod novo', 'CFOP','QTD_CAT', 'ICMS_TOT_PCAT', 'VLR_CONFR_PCAT', 'COD_LEGAL_PCAT', 'FONTE']

        df_1100 = planilha.copy()
        df_1100 = df_1100[cols_1100]
        df_1100['Data'] = df_1100['DATA']
        df_1100['DATA'] = df_1100['DATA'].astype(str)
        df_1100['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%d%m%Y') for x in df_1100['DATA']]
        #df_1100.rename(coluns={'Número Item': 'NUM_ITEM'}, inplace=True)


        df_1100['QTD_FIN'] = df_1100['QTD_CAT']
        df_1100['COD_REG'] = ['1100']*df_1100.shape[0]
        df_1100 = df_1100[['COD_REG','CHV_DOC','Data','DATA','NUM_ITEM','IND_OPER','cod novo','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT', 'FONTE']]
                        
        df_1100['Data'] = df_1100['Data'].astype(str)
        df_1100['Data'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m%Y') for x in df_1100['Data']]
                        
        df_1100['QTD_FIN'] = df_1100['QTD_FIN'].replace('nan', np.nan)
                        
        # Leitura do arquivo da 1050

        arquivo_1050 = pd.read_excel(os.path.join(pasta_loja, f'1050_loja_{loja}_p{periodo}_v1.xlsx'), dtype=str)
                                     
        mes_ano = []
        for date in arquivo_1050['REF']:
            mes_ano.append(str(date)[5:7] + str(date)[0:4])

        arquivo_1050['DATA'] = [datetime.strptime(x, '%Y%m').strftime('%m%Y') for x in arquivo_1050['REF'].astype(str)]
                                     
        # Definição de uma função para exportar os
        # dataframes de cada documentação em um
        # arquivo .txt com a formatação desejada

        def dataframe_to_txt_with_crlf(data, path):
            with open(path, 'a', newline='', encoding='utf-8') as f:
                for index, row in data.iterrows():
                    row_str = '|'.join(map(lambda x: '' if pd.isna(x) else str(x), row)) + '\r\n'
                    f.write(row_str)
                                     
        def format_number(value):
            locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # Set your desired locale
            formatted_value = locale.format('%.2f', value, grouping=True)
            formatted_value = str(formatted_value).replace('.', '')
            return formatted_value
                                     
        df_0150 = pd.read_excel(os.path.join(diretorio_referencias, f'0000_0150_P{periodo}.xlsx'), sheet_name='0150', dtype=str)
        df_0150 = df_0150[['LOJA', 'REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'COD_MUN', 'IE', 'DATA']]
                        
        
        df_0150 = df_0150[df_0150['LOJA'] == f'Loja {loja}']
        if df_0150.shape[0] == 0:
                mensagem = f'Loja {loja} não possui 0150. Favor atualizar arquivo'
                print(mensagem)
                with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                    arquivo.write(mensagem)
                sys.exit()
                                     
        df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
        df_0150['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0150['DATA']]
        df_0150['CNPJ'] = df_0150['CNPJ'].replace('', np.nan)
        df_0150['IE'] = df_0150['IE'].replace('', np.nan)
        df_0150['COD_PART'] = df_0150['COD_PART'].str.replace(r'^(FOR|CLI)', '', regex=True)
                                     
        df_0150 = df_0150[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'IE', 'COD_MUN', 'DATA']]
        df_0150 = df_0150[df_0150['CNPJ'].notnull()]
        df_0150 = df_0150.drop_duplicates()
        
        df_0150_final = pd.DataFrame()

        # Get the unique 'PERIODO' values
        unique_periodos = df_0150['DATA'].unique()

        # Iterate through unique 'PERIODO' values
        for valor_periodo in unique_periodos:
            # Filter the DataFrame for the current 'PERIODO' value
            filtered_df = df_0150[df_0150['DATA'] == valor_periodo]

            # Drop duplicates in the 'CNPJ' column for the current 'PERIODO'
            filtered_df = filtered_df.drop_duplicates(subset='CNPJ', keep='first')

            # Append the filtered results to the final DataFrame
            df_0150_final = pd.concat([df_0150_final,filtered_df])

        # Reset the index of the final DataFrame
        df_0150_final.reset_index(drop=True, inplace=True)
        df_0150_final = df_0150_final[df_0150_final['REG'] == '0150']
                                     
        df_0150_final = df_0150_final.drop_duplicates()
                                     
        df_cnpj_repetido = pd.DataFrame(columns=df_0150_final.columns)

        dates = list(df_0150_final['DATA'].unique())
        for date in dates:
            for cnpj in df_0150_final[df_0150_final['DATA'] == date]['CNPJ'].unique():
                if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]) > 1:
                    df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]
                    df_cnpj_repetido = pd.concat([df_cnpj_repetido, df])
                                     
        if df_cnpj_repetido.shape[0] > 0:
            with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                arquivo.write('Existe repetição de CNPJ. Favor verificar')
                arquivo.write(f'O CNPJ repetido é: {list(df_cnpj_repetido["CNPJ"].unique())}')
                arquivo.write(f'A data em que ocorreu a repetição é {list(df_cnpj_repetido["DATA"].unique())}')
            sys.exit()
                                     
                                     
        df_ie_repetido = pd.DataFrame(columns=df_0150_final.columns)

        dates = list(df_0150_final['DATA'].unique())
        for date in dates:
            for ie in df_0150_final[df_0150_final['DATA'] == date]['IE'].unique():
                if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]) > 1:
                    df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]
                    df_ie_repetido = pd.concat([df_ie_repetido, df])
                                     
        if df_ie_repetido.shape[0] > 0:
            with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                arquivo.write('Existe repetição de Inscrição Estadual. Favor verificar')
                arquivo.write(f'A IE repetido é: {list(df_ie_repetido["IE"].unique())}')
                arquivo.write(f'A data em que ocorreu a repetição é {list(df_ie_repetido["DATA"].unique())}')
            sys.exit()
                                     
        for cod_part in list(df_0150_final['COD_PART'].unique()):
            for date in df_0150_final['DATA']:
                if df_0150_final[(df_0150_final['COD_PART'] == cod_part) & (df_0150_final['DATA'] == date)].shape[0] > 1:
                    mensagem = 'ATENÇÃO. EXISTE CODIGO DE PARTICIPANTE REPETIDO. FAVOR CHECAR'
                    with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                        arquivo.write(mensagem)
                        arquivo.write(f'O codigo repetido é igual a {cod_part}')
                        arquivo.write(f'A data da repetição é igual a {date}')
                    sys.exit()
                       
        # Iterate through each file in the folder
        df_0000 = pd.read_excel(os.path.join(diretorio_referencias, f'0000_0150_P{periodo}.xlsx'), sheet_name='0000', dtype=str)

        df_0000['DATA'] = df_0000['DATA'].astype(str).str.slice(0, 8)
        df_0000['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0000['DATA']]
        df_0000['COD_VER'] = ['01'] * df_0000.shape[0]
        df_0000['COD_FIN'] = ['00'] * df_0000.shape[0]
        df_0000 = df_0000[['LOJA','REG', 'DATA', 'NOME', 'CNPJ', 'IE',
                'COD_MUN', 'COD_VER', 'COD_FIN']]
                                     
        df_0150_final['COD_MUN'] = pd.to_numeric(df_0150_final['COD_MUN'])
                                     
        def fill_missing_ie(group):
            non_null_ie = group.dropna()['IE']
            if not non_null_ie.empty:
                group['IE'] = non_null_ie.iloc[0]
            elif group['IE'].isnull().all():
                group['IE'] = np.nan
            return group

        # Group by 'CNPJ' and apply the custom function
        df_0150_final_new = df_0150_final.groupby('CNPJ', group_keys=False).apply(fill_missing_ie)
                                     
        df_0150_final_new = df_0150_final_new[df_0150_final_new['IE'].notnull()]
                                     
        if df_0150_final_new.loc[(df_0150_final_new['CNPJ'].notnull()) & (df_0150_final_new['IE'].isnull())].shape[0] > 0:
            with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                arquivo.write('Existe CNPJ sem Inscrição Estadual. Favor checar')
                arquivo.write(f'O CNPJ sem inscrição estadual é {list(df_0150_final_new.loc[(df_0150_final_new["CNPJ"].notnull()) & (df_0150_final_new["IE"].isnull())]["CNPJ"].unique())}')
            sys.exit()
        
        df_0150_final_new['CNPJ'] = df_0150_final_new['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))
                                     
        # Função para verificar o dígito verificador de um CNPJ
        def verifica_digito_verificador(cnpj):
            cnpj = [int(d) for d in str(cnpj) if d.isdigit()]  # Converte o CNPJ em uma lista de números inteiros
            if len(cnpj) != 14:
                return False  # CNPJ deve ter 14 dígitos

            # Calcula o primeiro dígito verificador
            soma = 0
            peso = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
            for i in range(12):
                soma += cnpj[i] * peso[i]
            resto = soma % 11
            if resto < 2:
                digito_verificador1 = 0
            else:
                digito_verificador1 = 11 - resto

            # Calcula o segundo dígito verificador
            soma = 0
            peso.insert(0, 6)
            for i in range(13):
                soma += cnpj[i] * peso[i]
            resto = soma % 11
            if resto < 2:
                digito_verificador2 = 0
            else:
                digito_verificador2 = 11 - resto

            # Verifica se os dígitos verificadores são iguais aos dígitos originais
            return cnpj[-2] == digito_verificador1 and cnpj[-1] == digito_verificador2

        # Exemplo de DataFrame
        df = df_0150_final_new.copy()

        # Aplica a função verifica_digito_verificador à coluna 'CNPJ' do DataFrame
        df['Dígito_Verificador_Válido'] = df['CNPJ'].apply(verifica_digito_verificador)
                                     
        if len(df['Dígito_Verificador_Válido'].unique()) > 1:
            with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
                arquivo.write('Existem CNPJs com dígito verificador inválido. Favor checar.')
                arquivo.writer(f'O CNPJ com dígito verificador inválido é {list(df["CNPJ"].unique())}')
            sys.exit()
          
        def recalcular_digito_verificador(nfe_completa):
            # Verifica se a NF-e com o dígito verificador tem o tamanho correto
            if pd.isnull(nfe_completa) or len(nfe_completa) != 44:
                raise ValueError(f"Valor inválido na coluna 'CHV_DOC': {nfe_completa}")

            # Remove o último caractere (dígito verificador)
            nfe_sem_dv = nfe_completa[:-1]

            # Restante da lógica de cálculo do dígito verificador
            soma = 0
            peso = 2
            nfe_invertida = nfe_sem_dv[::-1]

            for digito in nfe_invertida:
                soma += int(digito) * peso
                peso += 1
                if peso > 9:
                    peso = 2

            resto = soma % 11
            dv = 0 if resto == 0 or resto == 1 else 11 - resto

            return dv
        # Geração dos TXTs da CAT

        for date in planilha['Data'].unique():
        #for date in ['112019']:
            # Geração da 0000
            df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
            df_0000_['IE'] = df_0000_['IE'].str.replace('-', '')
            df_0000_ = df_0000_.drop('LOJA', axis=1)
            if df_0000_.shape[0] != 1:
                print(f'Erro no 0000 da loja {loja} para a data {date}. Favor verificar')
                sys.exit()

            # Geração da 0150

            # df_0150 has originally 8 columns
            df_0150_ = df_0150_final_new[df_0150_final_new['DATA'] == date]
            if df_0150_.shape[0] == 0:
                print(f'Loja {loja} não possui 0150. Favor atualizar arquivo')
                sys.exit()
            df_0150_ = df_0150_[df_0150_['CNPJ'].notnull()]
            df_0150_['IE'] = df_0150_['IE'].str.replace('-', '')
            df_0150_ = df_0150_[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF',
                        'IE', 'COD_MUN']]

            # Geração da 0200
            df_0200 = planilha[planilha['Data'] == date][['cod novo','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']].drop_duplicates()
            df_0200['COD_REG'] = ['0200']*(df_0200.shape[0])
            df_0200 = df_0200[['COD_REG', 'cod novo','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']]
            df_0200['ALIQUOTA'] = df_0200['ALIQUOTA'].astype(float).astype(int)
            df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].astype(str)
            df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].replace('nan', np.nan)
            df_0200.drop_duplicates(subset='cod novo', keep='first', inplace=True)
            for cod in list(df_0200['cod novo'].unique()):
                if df_0200[df_0200['cod novo'] == cod].shape[0] > 1:
                    mensagem = 'ATENÇÃO. EXISTE CODIGO DE PRODUTO DUPLICADO NO 0200.'
                    print(mensagem)
                    print(f'O código duplicado é {cod}')
                    sys.exit()
            # Geração da 1050
            unique_cod_items = planilha[planilha['Data'] == date]['cod novo'].unique()
            arquivo_1050['cod novo'] = arquivo_1050['cod novo'].astype(str)
            filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['cod novo'].isin(unique_cod_items)]
            filtered

            df_1050 = filtered[['cod novo','QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]

            df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
            df_1050['QTD_INI'] = df_1050['QTD_INI'].astype(float).astype(int).astype(str) + ',000'
            df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
            df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
            df_1050['SALDO_FINAL_MES_ICMS'] = df_1050['SALDO_FINAL_MES_ICMS'].astype(float).map(format_number)

            df_1050 = df_1050[['COD_REG', 'cod novo', 'QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
            df_1050.rename(columns={'COD_REG': 'REG'}, inplace=True)

            # Geração da 1100
            df_1100_ = df_1100[df_1100['Data'] == date]
            if df_1100_[df_1100_['VLR_CONFR_PCAT'] < 0.01].shape[0] > 0:
                mensagem = 'ATNEÇÂO. EXISTE VALOR DE CONFRONTO IGUAL A 0.'
                print(mensagem)
                sys.exit()
            df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
            df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
            df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
            df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(str).replace('nan', np.nan)
            df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','cod novo','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT', 'FONTE']]

            cnpj_loja = df_0000[df_0000['LOJA'] == f'Loja {loja}']['CNPJ'].values[0]
            df_1100_['CHV_DOC'] = np.where(df_1100_['FONTE'].isin(['entrada_oprprp', 'saida_nfe', 'dfe']),
                                   df_1100_['CHV_DOC'].str.slice(0,6) + cnpj_loja + df_1100_['CHV_DOC'].str.slice(20,44),
                                   df_1100_['CHV_DOC'])
            df_1100_['DV_NFe'] = df_1100_['CHV_DOC'].apply(recalcular_digito_verificador)
            df_1100_['CHV_DOC'] = df_1100_['CHV_DOC'].str.slice(0,-1) + df_1100_['DV_NFe'].astype(str)
            
            df_1100_['CNPJ'] = df_1100_['CHV_DOC'].str.slice(6,20)
            for cnpj in df_1100_['CNPJ'].unique():
                if df_0150_[df_0150_['CNPJ'] == cnpj].shape[0] == 0:
                    mensagem = 'ATENÇÃO. EXISTE PARTICIPANTE NO 1100 QUE NÃO ESTÁ NO 0150.'
                    print(mensagem)
                    print(f'O participante faltante no 0150 tem CNPJ {cnpj}, para o mês {date}')
                    sys.exit()

            df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','cod novo','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]

            # Geração do .txt da 0000
            dataframe_to_txt_with_crlf(df_0000_, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\0000_{date}_{loja}_v1.txt')

            # Geração do .txt da 0150
            dataframe_to_txt_with_crlf(df_0150_, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\0150_{date}_{loja}_v1.txt')

            # Geração do .txt da 0200
            dataframe_to_txt_with_crlf(df_0200, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\0200_{date}_{loja}_v1.txt')

            # Geração do .txt da 1050
            dataframe_to_txt_with_crlf(df_1050, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\1050_{date}_{loja}_v1.txt')

            # Geração do .txt da 1100
            dataframe_to_txt_with_crlf(df_1100_, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\1100_{date}_{loja}_v1.txt')

            # Geração de um arquivo .txt único,
            # contendo todas as informações das documentações
            # da 0000, 0150, 0200, 1050 e 1100
            # dentro do formato correto

            read_files = glob.glob(f"C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\*{date}_{loja}_v1.txt")

            with open(f"C:\\Bots\\Bot_transmissao\\Geracao CAT\\cat_Loja_{loja}_{date}_v1.txt", "wb") as outfile:
                for f in read_files:
                    with open(f, "rb") as infile:
                        outfile.write(infile.read())
                                     
        #####################################################################################################################                        

        # Iterar sobre os arquivos na pasta da loja
        for arquivo_txt in os.listdir(pasta_loja):
            if arquivo_txt.startswith(f"cat_Loja_{loja}_") and arquivo_txt.endswith("_v1.txt"):
                # Construir o caminho completo para o arquivo de texto
                caminho_txt = os.path.join(pasta_loja, arquivo_txt)

                # Mover o arquivo para a pasta 'Geração CAT'
                destino_arquivo_txt = os.path.join(diretorio_principal)
                shutil.move(caminho_txt, destino_arquivo_txt)

        print(f"Arquivos movidos com sucesso para 'Geração CAT'")
    else:
        print(f"Pasta da loja não encontrada para o número {loja}")

# Diretório principal
diretorio_principal = "C:\\Bots\\Bot_transmissao\\Geracao CAT"

# Diretório de referencias
diretorio_referencias = os.path.join(diretorio_principal,'Referencias')

# Solicitar input para o nome do arquivo
nome_arquivo = input("Digite o nome do arquivo: ")

# Chamar a função para mover os arquivos
try:
    processar_loja(nome_arquivo, diretorio_principal)
except Exception as e:
    loja = nome_arquivo.split('_')[1]
    periodo = nome_arquivo.split('_')[4][1]
    # Criar um log com a mensagem de erro
    with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as log_file:
        log_file.write(f"Erro durante a execução: {str(e)}\n")
        log_file.write(traceback.format_exc())  # Adiciona a stack trace no log
        
end = time.time()

print(end - start)

Digite o nome do arquivo: loja_67_tabela_2_p2_v1
C:\Bots\Bot_transmissao\Geracao CAT\Loja_67


C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:150: DtypeWarning: Columns (5,7,22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(pasta_loja, f'loja_{loja}_tabela_2_1_p{periodo}_v1.csv'), encoding='utf-8')
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DATA'] = pd.to_datetime(data['DATA'], format='%Y-%m-%d')
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1183: SettingWithCopyWarning: 
A value i

C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1183: SettingWithCopyWarning: 
A value i

C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:930: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  formatted_value = locale.format('%.2f', value, grouping=True)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

...

C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:930: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  formatted_value = locale.format('%.2f', value, grouping=True)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['SALDO_FINAL_MES_ICMS'] = df_1050['SALDO_FINAL_MES_ICMS'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:930: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  formatted_value = locale.format('%.2f', value, grouping=True)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:930: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  formatted_value = locale.format('%.2f', value, grouping=True)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_21200\3349798297.py:1183: SettingWithCopyWarning: 
A value i

Arquivos movidos com sucesso para 'Geração CAT'
13898.352982759476
